In [21]:
pip install pygame


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import numpy as np
import pygame
import math
import random
from IPython.display import display, HTML

<h2> Parameters </h2>

In [23]:
SIDE_LEN = 11
START_SQAURE = [5, 5]
GOAL_SQAURE = [1, 1]
REWARD_INDEX = 100
LEARNING_RATE = .9
DISCOUNT_FACTOR = .75
MAX_RUNTIME = SIDE_LEN**2


<H1> Initialize Environment </h1>
Set up the board the agent will work with, plus the randomized start/finish blocks

In [24]:
gameBoard = [[0 for i in range(SIDE_LEN)] for j in range(SIDE_LEN)]
rewardBoard = [[0 for i in range(SIDE_LEN)] for j in range(SIDE_LEN)]



CurrSquare = START_SQAURE.copy()

def upMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[0] == 0:
        return
    curr[0] = curr[0]-1
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward
    
def downMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[0] == SIDE_LEN-1:
        return
    curr[0] = curr[0]+1
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward
    
def leftMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[1] == 0:
        return
    curr[1] = curr[1]-1
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward

def rightMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[1] == SIDE_LEN-1:
        return
    curr[1] = curr[1]+1   
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward

def calcDist(curr):
    one = curr[0] - GOAL_SQAURE[0]
    two = curr[1] - GOAL_SQAURE[1]
    return math.sqrt(pow(one, 2) + pow(two, 2))


START_BASE_REWARD = round(REWARD_INDEX/calcDist(START_SQAURE), 2)

def calcReward(curr):
    if curr == GOAL_SQAURE:
        return REWARD_INDEX**2
    else:
        distance = calcDist(curr)
        reward = round(REWARD_INDEX/distance, 2) - START_BASE_REWARD
        reward = reward*REWARD_INDEX
        return round(reward, 2)



In [25]:


# upMove(CurrSquare)

# for row in gameBoard:
#     print(row)
    
# print("")
# print(" - - - - - - - - - - - ")
# print("")

# for row in rewardBoard:
#     print(row)
    
# print("")
# print(" - - - - - - - - - - - ")
# print("")

# print(CurrSquare)

<h1> Initialize SAV Table and Q-Table</h1>

State-Action-Value table: <br>
    Each Row is a state<br>
    Each Column is a different action<br>
    Each Cell is the reard for taking that columns action after starting in the row's state<br>
    Final Column is Best Action<br>
    

In [26]:
SAV = pd.DataFrame(-1.0, index = np.arange(SIDE_LEN**2), columns=["UP", "DOWN", "LEFT", "RIGHT"])
SAV['MAX'] = 0.0
TOP_SAV = pd.DataFrame('na', index = np.arange(SIDE_LEN**2), columns = ['BEST'])
TOP_SAV['BIDX'] = -1

def updateSAV(prev, move, reward, curr):
    idx = prev[0]*11 + prev[1]
    curridx = curr[0]*11 + curr[1]
    SAV.at[idx, move] = reward
    SAV['MAX'] = SAV.max(axis=1)
    max_action = SAV.loc[idx, ['UP', 'DOWN', 'LEFT', 'RIGHT']].idxmax()
    TOP_SAV.at[idx, 'BEST'] = max_action
    TOP_SAV.at[idx, 'BIDX'] = calc_Max_Idx(idx, max_action)
    
def calc_Max_Idx(currIdx, action):
    if action == 'UP':
        maxidx = currIdx - SIDE_LEN
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx
    
    if action == 'DOWN':
        maxidx = currIdx + SIDE_LEN
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx
    
    if action == 'LEFT':
        maxidx = currIdx - 1
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx
    
    if action == 'RIGHT':
        maxidx = currIdx + 1
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx

        
Q_TABLE = pd.DataFrame(0.0, index = np.arange(SIDE_LEN**2), columns=["UP", "DOWN", "LEFT", "RIGHT", "MAX"])
TOP_Q = pd.DataFrame('na', index = np.arange(SIDE_LEN**2), columns = ['BEST'])
TOP_Q['BIDX'] = -1

def bellmanEquation(currQ, reward, stateIdx, df, recurse):
    if df.at[stateIdx, 'BEST'] == 'na':
        return currQ + (LEARNING_RATE * (reward + DISCOUNT_FACTOR * (0))) - currQ
    else:
        if recurse == 20:
            return currQ + (LEARNING_RATE * (reward + DISCOUNT_FACTOR * (0))) - currQ
        recurse += 1
        nextIdx = df.at[stateIdx, 'BIDX']
        nextQ = Q_TABLE.at[nextIdx, 'MAX']
        nextReward = SAV.at[nextIdx, 'MAX']

        return currQ +(LEARNING_RATE * (reward + DISCOUNT_FACTOR * (bellmanEquation(nextQ, nextReward, nextIdx, df, recurse)))) - currQ
    
def updateQTable(prev, move, reward, curr):
    idx = prev[0]*11 + prev[1]
    curridx = curr[0]*11 + curr[1]
    if idx != TOP_Q.at[curridx, 'BIDX']:
        Q_TABLE.at[idx, move] = bellmanEquation(Q_TABLE.at[idx, move], reward, idx, TOP_Q, 0)
    Q_TABLE['MAX'] = Q_TABLE.max(axis=1)
    max_action = Q_TABLE.loc[idx, ['UP', 'DOWN', 'LEFT', 'RIGHT']].idxmax()
    TOP_Q.at[idx, 'BEST'] = max_action
    TOP_Q.at[idx, 'BIDX'] = calc_Max_Idx(idx, max_action)

    

In [27]:
def move(direction, curr):
    prev = curr.copy()
    match direction:
        case "UP":
            upMove(curr)
        case "DOWN":
            downMove(curr)
        case "LEFT":
            leftMove(curr)
        case "RIGHT":
            rightMove(curr)
        case _:
            print("INVALID MOVE ERROR")
            exit(99)
    if curr == prev:
        reward = -1 * (REWARD_INDEX**2)
    else:
        reward = rewardBoard[curr[0]][curr[1]]
    # print(prev)
    # print(curr)
    updateSAV(prev, direction, reward, curr)
    updateQTable(prev, direction, reward, curr)
    

# move("UP", CurrSquare)
# print(SAV.loc[60,:])
# print(TOP_SAV.loc[60,:])
# print(Q_TABLE.loc[60,:])
# print(TOP_Q.loc[60,:])
    

<h1> Agent Time </h1>

In [28]:
def runAgent(curr, iter, randChance):
    moves = 0
    while moves < MAX_RUNTIME:
        rand = random.randint(1,100)
        currIdx = curr[0]*11 + curr[1]
        if rand <= randChance:
            direct = SAV.loc[currIdx][SAV.loc[currIdx] == -1.0].index.to_list()
            # direct = ["UP", "DOWN", "LEFT", "RIGHT"]
            if len(direct) == 0:
                direction = TOP_Q.at[currIdx, 'BEST']
                if direction == 'na':
                    direction = random.choice(["UP", "DOWN", "LEFT", "RIGHT"])
            else:
                direction = random.choice(direct)
        else:
            
            direction = TOP_Q.at[currIdx, 'BEST']
            if direction == 'na':
                direct = SAV.loc[currIdx][SAV.loc[currIdx] == -1.0].index.to_list()
                if len(direct) == 0:
                    direction = random.choice(["UP", "DOWN", "LEFT", "RIGHT"])
                else:
                    direction = random.choice(direct)
        prev = curr.copy()
        if iter >= 100:
            print(f"Iter: {iter} - Move {moves} - Curr: {curr}")
        move(direction, curr)
        if curr == prev:
            print("Hit an edge")
            break
        if curr == GOAL_SQAURE:
            print("REACHED GOAL!")
            break
        moves += 1
        
    print(f"Iter: {iter} - Stopped after {moves} moves.")
    print("") 

In [29]:
MAX_ITER = 110
CurrSqaure = START_SQAURE

for i in range(MAX_ITER):
    runAgent(CurrSquare, i, 100-1)
    gameBoard = [[0 for i in range(SIDE_LEN)] for j in range(SIDE_LEN)]
    CurrSquare = START_SQAURE.copy()
    


Hit an edge
Iter: 0 - Stopped after 65 moves.

Hit an edge
Iter: 1 - Stopped after 70 moves.

Hit an edge
Iter: 2 - Stopped after 10 moves.



REACHED GOAL!
Iter: 3 - Stopped after 19 moves.

Hit an edge
Iter: 4 - Stopped after 8 moves.

REACHED GOAL!
Iter: 5 - Stopped after 7 moves.

REACHED GOAL!
Iter: 6 - Stopped after 7 moves.

REACHED GOAL!
Iter: 7 - Stopped after 7 moves.

REACHED GOAL!
Iter: 8 - Stopped after 7 moves.

REACHED GOAL!
Iter: 9 - Stopped after 7 moves.

REACHED GOAL!
Iter: 10 - Stopped after 7 moves.

REACHED GOAL!
Iter: 11 - Stopped after 7 moves.

REACHED GOAL!
Iter: 12 - Stopped after 7 moves.

REACHED GOAL!
Iter: 13 - Stopped after 7 moves.

REACHED GOAL!
Iter: 14 - Stopped after 7 moves.

REACHED GOAL!
Iter: 15 - Stopped after 7 moves.

REACHED GOAL!
Iter: 16 - Stopped after 7 moves.

REACHED GOAL!
Iter: 17 - Stopped after 7 moves.

REACHED GOAL!
Iter: 18 - Stopped after 7 moves.

REACHED GOAL!
Iter: 19 - Stopped after 7 moves.

REACHED GOAL!
Iter: 20 - Stopped after 7 moves.

REACHED GOAL!
Iter: 21 - Stopped after 7 moves.

REACHED GOAL!
Iter: 22 - Stopped after 7 moves.

REACHED GOAL!
Iter: 23 - Sto

In [30]:
display(HTML(TOP_Q.to_html()))
for row in rewardBoard:
    print(row)


[0, 0, 5303.0, 0, 1394.0, 0, 0, -124.0, 0, 0, 0]
[0, 10000, 8232.0, 3232.0, 1565.0, 732.0, 232.0, -101.0, -339.0, 0, 0]
[5303.0, 8232.0, 5303.0, 2704.0, 1394.0, 657.0, 193.0, -124.0, -354.0, -528.0, 0]
[0, 3232.0, 2704.0, 1768.0, 1006.0, 468.0, 89.0, -187.0, 0, -555.0, 0]
[0, 0, 1394.0, 1006.0, 589.0, 232.0, -53.0, -277.0, -455.0, -598.0, -714.0]
[0, 732.0, 657.0, 468.0, 232.0, 0.0, -206.0, -381.0, -528.0, -650.0, 0]
[0, 232.0, 193.0, 89.0, -53.0, -206.0, -354.0, -488.0, 0, 0, 0]
[0, 0, -124.0, -187.0, -277.0, -381.0, -488.0, -589.0, 0, 0, 0]
[0, 0, 0, 0, -455.0, -528.0, -606.0, -683.0, 0, 0, 0]
[0, 0, 0, 0, -598.0, -650.0, -708.0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, -753.0, 0, 0, 0, 0, 0]


In [31]:
display(HTML(Q_TABLE.sort_values('MAX', ascending=False).head(100).to_html()))